# Writing PRISM

In [1]:
from stormvogel import visualization, mapping, result, show
import stormpy

Using the ``%%prism`` keyword, you can immediately tinker with PRISM programs in the notebooks. This example is the [NAND multiplexing case study from the PRISM website](https://www.prismmodelchecker.org/casestudies/nand.php). If you provide a variable name after the keyword, this variable will store your PRISM program after the execution of the cell.

In [2]:
%%prism nand
dtmc

const int N = 2; // number of inputs in each bundle
const int K = 2; // number of restorative stages

const int M = 2*K+1; // total number of multiplexing units

const double perr = 0.02; // probability nand works correctly
const double prob1 = 0.9; // probability initial inputs are stimulated

module multiplex

	u : [1..M]; // number of stages
	c : [0..N]; // counter (number of copies of the nand done)

	s : [0..4]; // local state
	// 0 - initial state
	// 1 - set x inputs
	// 2 - set y inputs
	// 3 - set outputs
	// 4 - done

	z : [0..N]; // number of new outputs equal to 1
	zx : [0..N]; // number of old outputs equal to 1
	zy : [0..N]; // need second copy for y
	// initially 9 since initially probability of stimulated state is 0.9

	x : [0..1]; // value of first input
	y : [0..1]; // value of second input
	
	[] s=0 & (c<N) -> (s'=1); // do next nand if have not done N yet
	[] s=0 & (c=N) & (u<M) -> (s'=1) & (zx'=z) & (zy'=z) & (z'=0) & (u'=u+1) & (c'=0); // move on to next u if not finished
	[] s=0 & (c=N) & (u=M) -> (s'=4) & (zx'=0) & (zy'=0) & (x'=0) & (y'=0); // finished (so reset variables not needed to reduce state space)

	// choose x permute selection (have zx stimulated inputs)
	// note only need y to be random	
	[] s=1 & u=1  -> prob1 : (x'=1) & (s'=2) + (1-prob1) : (x'=0) & (s'=2); // initially random
	[] s=1 & u>1 & zx>0 -> (x'=1) & (s'=2) & (zx'=zx-1);
	[] s=1 & u>1 & zx=0 -> (x'=0) & (s'=2);

	// choose x randomly from selection (have zy stimulated inputs)
	[] s=2 & u=1 -> prob1 : (y'=1) & (s'=3) + (1-prob1) : (y'=0) & (s'=3); // initially random
	[] s=2 & u>1 & zy<(N-c) & zy>0  -> zy/(N-c) : (y'=1) & (s'=3) & (zy'=zy-1) + 1-(zy/(N-c)) : (y'=0) & (s'=3);
	[] s=2 & u>1 & zy=(N-c) & c<N -> 1 : (y'=1) & (s'=3) & (zy'=zy-1);
	[] s=2 & u>1 & zy=0 -> 1 : (y'=0) & (s'=3);

	// use nand gate
	[] s=3 & z<N & c<N -> (1-perr) : (z'=z+(1-x*y)) & (s'=0) & (c'=c+1) & (x'=0) & (y'=0) // not faulty
	         + perr    : (z'=z+(x*y))    & (s'=0) & (c'=c+1) & (x'=0) & (y'=0); // von neumann fault
	
	[] s=4 -> (s'=s);
	
endmodule

// rewards: final value of gate
rewards
	// [] s=0 & (c=N) & (u=M) : z/N;
	s=0 & (c=N) & (u=M) : z/N;
endrewards

label "nextnand" = s=0;
label "selection" = s=1;
label "whichx" = s=2;
label "gate" = s=3 & z<N & c<N;
label "target" = s=4 & z/N<0.1;
label "end" = s=4;


We now have a PRISM program, but that is not a Markov model yet, only a description. We can build this into a full model using stormpy:

In [3]:
nand_model = stormpy.build_model(nand)

If we want to do model checking later, we need a property as well, such as:

In [4]:
prop = stormpy.parse_properties("P=? [F \"target\"]", nand)

We can print the model to get some basic information.

In [5]:
print(nand_model)

-------------------------------------------------------------- 
Model type: 	DTMC (sparse)
States: 	178
Transitions: 	243
Reward Models:  (default)
State Labels: 	8 labels
   * init -> 1 item(s)
   * whichx -> 42 item(s)
   * selection -> 39 item(s)
   * nextnand -> 42 item(s)
   * deadlock -> 0 item(s)
   * target -> 1 item(s)
   * gate -> 52 item(s)
   * end -> 3 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 



In [6]:
stormpy_result = stormpy.model_checking(nand_model, prop[0])

In [7]:
print(stormpy_result.get_values())

[0.7434211305545204, 0.7434211305545204, 0.7742858388475886, 0.46563875591690623, 0.80857995917322, 0.46563875591690623, 0.46563875591690623, 0.46563875591690623, 0.8157245675743932, 0.458494147515733, 0.8157245675743932, 0.458494147515733, 0.8426262463942631, 0.573609458195563, 0.504976011479863, 0.04015737183856233, 0.8725170006385631, 0.573609458195563, 0.573609458195563, 0.573609458195563, 0.556622526995563, 0.04015737183856233, 0.04015737183856233, 0.04015737183856233, 0.8787442411061255, 0.5673822177280005, 0.029397681106124816, 0.8787442411061255, 0.5673822177280005, 0.029397681106124816, 0.8787442411061255, 0.5673822177280005, 0.029397681106124816, 0.8787442411061255, 0.2560201943498754, 0.8787442411061255, 0.029397681106124816, 0.8917176587468806, 0.24304677670912014, 0.8917176587468806, 0.24304677670912014, 0.02467637874688001, 0.26074149670912017, 0.8917176587468806, 0.24304677670912014, 0.8917176587468806, 0.24304677670912014, 0.02467637874688001, 0.26074149670912017, 0.891

Let's visualize the model in an interactive graph :)

In [8]:
stormvogel_model = mapping.stormpy_to_stormvogel(nand_model)

stormvogel_result = result.convert_model_checking_result(stormvogel_model, stormpy_result)

In [9]:
import stormvogel.layout

vis = show.show(stormvogel_model, stormvogel_result, name="prism", layout=stormvogel.layout.EXPLORE(), show_editor=False)